<a href="https://colab.research.google.com/github/Rudrajit12/Retail-Sales-Performance-Analysis/blob/main/Retail_Sales_Performance_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Problem Statement**:
XYZ Retail, a leading chain of stores across major US cities, has been experiencing inconsistencies in its sales reporting system. The company operates in various sectors, including Electronics, Clothing, Furniture, Toys, and Groceries, with stores located in bustling urban areas such as New York, Los Angeles, Chicago, Houston, and Phoenix.

Over the past year, the company's data management has suffered from lapses, resulting in missing information, outliers, and formatting inconsistencies in its sales records. These issues have made it difficult for the leadership team to assess the business's true performance, identify top-selling products, understand customer purchasing behavior, and compare the performance of stores across regions.

### **Business Scenario**:
The management has tasked the data analytics team with building an end-to-end solution that not only provides insights into sales performance but also focuses on resolving the data quality issues present in the system. Specifically, the team is required to:
1. **Clean and preprocess the data** by addressing missing values, handling inconsistencies, and dealing with outliers.
2. **Create a dashboard** that provides insights into the following:
   - Sales trends over time.
   - The performance of various product categories.
   - Customer segmentation based on purchasing behavior.
   - Regional performance of stores.
   - Identifying any seasonal trends in product sales or customer activity.

### **Challenges**:
1. **Missing Data**: Due to lapses in data entry, some sales records have incomplete information, including missing prices, discounts, and store locations. The team must find ways to impute or handle this missing data without skewing the results.
   
2. **Inconsistent Data**: The sales data also includes inconsistencies in fields like `Store Location` (marked as "Unknown"), `Gender` (some entries labeled "Other"), and `Region` (with "InvalidRegion" entries). These need to be standardized.

3. **Outliers**: A few extreme values in the `Quantity` and `Total Sale` columns could distort the analysis if not addressed properly.

4. **Multiple Data Sources**: The dataset consists of three key data tables — `Sales Data`, `Customer Data`, and `Product Data`. These need to be integrated and cross-referenced to provide a unified view of sales performance.

### **Goal**:
The main goal of the project is to clean the dataset, perform an exploratory data analysis (EDA), and create a visually informative dashboard. This dashboard will help management make informed decisions regarding:
- Stock replenishment for top-selling products.
- Regional marketing campaigns.
- Targeted promotions based on customer segments.
- Improving customer experience through personalized offers.



#### **Load the dataset and understand the variables present in the dataset:**

In [1]:
# Loading and importing the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

plt.rcParams['figure.figsize'] = [13,9]
sns.set(rc={'figure.figsize':(13,9)})

In [2]:
# Mount the drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [3]:
# Loading the datasets
df_customer = pd.read_csv('/content/drive/My Drive/Retail Sales Performance Analysis/Customer_Data.csv')
df_product = pd.read_csv('/content/drive/My Drive/Retail Sales Performance Analysis/Product_Data.csv')
df_sales = pd.read_csv('/content/drive/My Drive/Retail Sales Performance Analysis/Sales_Data.csv')

In [4]:
# Display the datasets
df_customer.head()

,Unnamed: 0,Customer ID,Name,Age,Gender,Segment,Region
0,0,C001,Nicole Williams,64.0,Male,Loyal,West
1,1,C002,Carl Mitchell,23.0,Non-binary,Loyal,South
2,2,C003,Nichole White,66.0,Male,Loyal,West
3,3,C004,Jasmine Sheppard,35.0,Non-binary,Returning,East
4,4,C005,Dr. Jason Jones,58.0,Non-binary,Loyal,West


In [5]:
# Display the datasets
df_product.head()

,Unnamed: 0,Product ID,Product Name,Category,Price,Cost
0,0,P001,Organization,Furniture,33.81,64.92
1,1,P002,Form,Clothing,326.78,154.28
2,2,P003,Enough,Furniture,423.45,108.41
3,3,P004,First,Furniture,394.90,152.41
4,4,P005,Evening,Toys,377.35,194.77


In [6]:
# Display the datasets
df_sales.head()

,Unnamed: 0,Date,Order ID,Customer ID,Product ID,Quantity,Price,Discount,Store Location,Total Sale
0,0,2024-04-22,O0001,C487,P021,4,114.79,0.11,Chicago,408.65
1,1,2024-01-08,O0002,C412,P070,3,62.82,0.19,Houston,152.65
2,2,2024-04-12,O0003,C502,P048,6,418.02,0.00,Houston,2508.12
3,3,2024-02-02,O0004,C154,P024,7,327.72,0.24,Chicago,1743.47
4,4,2023-09-26,O0005,C374,P091,2,404.10,0.10,New York,727.38


### **Data Description:**

The project involves three interconnected datasets: **Sales Data**, **Customer Data**, and **Product Data**. These datasets contain key business metrics required to evaluate XYZ Retail's performance over the past year. Here's a breakdown of the data:

---

#### **1. Sales Data**:
This dataset captures individual sales transactions across different store locations.

| **Column**       | **Description**                                                                                   |
|------------------|---------------------------------------------------------------------------------------------------|
| **Date**         | The date on which the sales transaction occurred.                                                  |
| **Order ID**     | Unique identifier for each sales order.                                                            |
| **Customer ID**  | Unique identifier for the customer who made the purchase.                                          |
| **Product ID**   | Unique identifier for the product sold.                                                            |
| **Quantity**     | The number of units of the product sold in the transaction.                                         |
| **Price**        | The price per unit of the product (in some cases missing).                                          |
| **Discount**     | Discount applied to the order (values range from 0% to 30%).                                        |
| **Store Location**| The store where the sale occurred (contains missing and inconsistent values).                      |
| **Total Sale**   | The total sales value after applying the discount (`Price * Quantity * (1 - Discount)`).            |



---

#### **2. Customer Data**:
This dataset contains demographic and segment information about the customers making purchases.

| **Column**     | **Description**                                                                                   |
|----------------|---------------------------------------------------------------------------------------------------|
| **Customer ID**| Unique identifier for each customer.                                                              |
| **Name**       | The name of the customer.                                                                          |
| **Age**        | The age of the customer (contains missing values).                                                 |
| **Gender**     | Gender of the customer (values include 'Male', 'Female', 'Non-binary', and inconsistent 'Other').  |
| **Segment**    | Customer segment (e.g., 'Loyal', 'Returning', 'New') based on their shopping behavior.             |
| **Region**     | The region where the customer is located (contains invalid entries such as "InvalidRegion").       |



---

#### **3. Product Data**:
This dataset provides details about the products sold across different categories.

| **Column**      | **Description**                                                                             |
|-----------------|---------------------------------------------------------------------------------------------|
| **Product ID**  | Unique identifier for each product.                                                         |
| **Product Name**| The name of the product.                                                                    |
| **Category**    | The category of the product (e.g., Electronics, Clothing, Furniture, Toys, Groceries).      |
| **Price**       | The price per unit of the product (used to calculate `Total Sale` in the Sales Data).       |
| **Cost**        | The cost price of the product, indicating the expense incurred by the business.             |


---



In [10]:
# Check data types of the variables in these datasets
print('Customer Data:')
print(df_customer.dtypes)
print('--------------')
print('Product Data')
print(df_product.dtypes)
print('------------')
print('Sales Data')
print(df_sales.dtypes)

Customer Data:
Unnamed: 0       int64
Customer ID     object
Name            object
Age            float64
Gender          object
Segment         object
Region          object
dtype: object
--------------
Product Data
Unnamed: 0        int64
Product ID       object
Product Name     object
Category         object
Price           float64
Cost            float64
dtype: object
------------
Sales Data
Unnamed: 0          int64
Date               object
Order ID           object
Customer ID        object
Product ID         object
Quantity            int64
Price             float64
Discount          float64
Store Location     object
Total Sale        float64
dtype: object


In [13]:
# Number of records present in each dataset
print('Customer Data:', len(df_customer))
print('Product Data:', len(df_product))
print('Sales Data:', len(df_sales))

Customer Data: 1000
Product Data: 100
Sales Data: 1000000


#### **Check for inconsistencies in the dataset:**

In [15]:
# Check for duplicates in the dataset
print('Customer Data:', df_customer.duplicated().sum())
print('Product Data:', df_product.duplicated().sum())
print('Sales Data:', df_sales.duplicated().sum())

Customer Data: 0
Product Data: 0
Sales Data: 0


In [17]:
# Check for missing values in the dataset
print('Customer Data:', df_customer.isnull().sum())
print('----------------------')
print('Product Data:', df_product.isnull().sum())
print('----------------------')
print('Sales Data:', df_sales.isnull().sum())

Customer Data: Unnamed: 0      0
Customer ID     0
Name            0
Age            10
Gender          0
Segment         0
Region          0
dtype: int64
----------------------
Product Data: Unnamed: 0      0
Product ID      0
Product Name    1
Category        0
Price           0
Cost            0
dtype: int64
----------------------
Sales Data: Unnamed: 0         0
Date               0
Order ID           0
Customer ID        0
Product ID         0
Quantity           0
Price             30
Discount          25
Store Location    20
Total Sale         0
dtype: int64


#### **Investigating each data table separately:**

In [24]:
# Check info for customer table
df_customer.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Unnamed: 0   1000 non-null   int64  
 1   Customer ID  1000 non-null   object 
 2   Name         1000 non-null   object 
 3   Age          990 non-null    float64
 4   Gender       1000 non-null   object 
 5   Segment      1000 non-null   object 
 6   Region       1000 non-null   object 
dtypes: float64(1), int64(1), object(5)
memory usage: 54.8+ KB


In [27]:
# Check contents of each category
print(df_customer['Gender'].value_counts())
print(df_customer['Segment'].value_counts())
print(df_customer['Region'].value_counts())

Gender
Non-binary    350
Male          327
Female        318
Other           5
Name: count, dtype: int64
Segment
Returning    346
New          333
Loyal        321
Name: count, dtype: int64
Region
East             209
West             201
North            201
South            200
Central          186
InvalidRegion      3
Name: count, dtype: int64


In [28]:
# Check info for product table
df_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Unnamed: 0    100 non-null    int64  
 1   Product ID    100 non-null    object 
 2   Product Name  99 non-null     object 
 3   Category      100 non-null    object 
 4   Price         100 non-null    float64
 5   Cost          100 non-null    float64
dtypes: float64(2), int64(1), object(3)
memory usage: 4.8+ KB


In [32]:
# Check content for each category
print(df_product['Category'].value_counts())

Category
Groceries      26
Clothing       21
Toys           21
Electronics    18
Furniture      14
Name: count, dtype: int64


In [33]:
# Check info for sales table
df_sales.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000000 entries, 0 to 999999
Data columns (total 10 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   Unnamed: 0      1000000 non-null  int64  
 1   Date            1000000 non-null  object 
 2   Order ID        1000000 non-null  object 
 3   Customer ID     1000000 non-null  object 
 4   Product ID      1000000 non-null  object 
 5   Quantity        1000000 non-null  int64  
 6   Price           999970 non-null   float64
 7   Discount        999975 non-null   float64
 8   Store Location  999980 non-null   object 
 9   Total Sale      1000000 non-null  float64
dtypes: float64(3), int64(2), object(5)
memory usage: 76.3+ MB


In [34]:
# Check content for each category
print(df_sales['Store Location'].value_counts())

Store Location
Chicago        200276
Los Angeles    200269
Houston        200242
New York       200149
Phoenix        199039
Unknown             5
Name: count, dtype: int64


### **Dataset Summary**:

1. **Sales Data**: Contains 1,00,000 sales transactions over the past year across multiple stores. This dataset includes key transaction information but has missing and inconsistent values that require cleaning.

2. **Customer Data**: Covers 1000 unique customers with demographic and segment information. The data needs to be cleaned to handle missing and inconsistent values in fields like `Age`, `Gender`, and `Region`.

3. **Product Data**: Contains 100 unique products across various categories, with pricing and cost information. It provides the backbone for calculating sales performance and margins.

---